# Clustering Crypto

In [30]:
# Initial imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [2]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
response = requests.get(url)

In [3]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
res_js = response.json()
crypto_df = pd.DataFrame.from_dict(res_js['Data'],orient="index")
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,AlgorithmType,Difficulty,BuiltOn,SmartContractAddress,DecimalPoints
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,42.0,0.0,0.0,0.0,blockchain,scrypt,0.000244,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300.0,0.0,0.0,0.0,token,NaN,NaN,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18.0
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,-1.0,0.0,0.0,0.0,blockchain,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,-1.0,0.0,0.0,0.0,blockchain,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame

### Data Preprocessing

In [5]:
crypto_df.columns

Index(['Id', 'Url', 'ImageUrl', 'ContentCreatedOn', 'Name', 'Symbol',
       'CoinName', 'FullName', 'Description', 'AssetTokenStatus', 'Algorithm',
       'ProofType', 'SortOrder', 'Sponsored', 'Taxonomy', 'Rating',
       'IsTrading', 'TotalCoinsMined', 'CirculatingSupply', 'BlockNumber',
       'NetHashesPerSecond', 'BlockReward', 'BlockTime', 'AssetLaunchDate',
       'AssetWhitepaperUrl', 'AssetWebsiteUrl', 'MaxSupply', 'MktCapPenalty',
       'IsUsedInDefi', 'IsUsedInNft', 'PlatformType', 'AlgorithmType',
       'Difficulty', 'BuiltOn', 'SmartContractAddress', 'DecimalPoints'],
      dtype='object')

In [6]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','CirculatingSupply']]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,CirculatingSupply
42,42 Coin,Scrypt,True,PoW/PoS,41.999952,41.999952
300,300 token,N/A,True,N/A,300,0.000000
365,365Coin,X11,True,PoW/PoS,0,0.000000
404,404Coin,Scrypt,True,PoW/PoS,0,0.000000
433,433 Token,N/A,False,N/A,NaN,NaN


In [7]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,CirculatingSupply
42,42 Coin,Scrypt,True,PoW/PoS,41.999952,41.999952
300,300 token,N/A,True,N/A,300,0.000000
365,365Coin,X11,True,PoW/PoS,0,0.000000
404,404Coin,Scrypt,True,PoW/PoS,0,0.000000
611,SixEleven,SHA-256,True,PoW,0,0.000000


In [8]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,CirculatingSupply
42,42 Coin,Scrypt,True,PoW/PoS,41.999952,41.999952
365,365Coin,X11,True,PoW/PoS,0,0.000000
404,404Coin,Scrypt,True,PoW/PoS,0,0.000000
611,SixEleven,SHA-256,True,PoW,0,0.000000
808,808,SHA-256,True,PoW/PoS,0,0.000000


In [9]:
# Remove the "IsTrading" column
crypto_df.drop(columns=['IsTrading'],inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,CirculatingSupply
42,42 Coin,Scrypt,PoW/PoS,41.999952,41.999952
365,365Coin,X11,PoW/PoS,0,0.000000
404,404Coin,Scrypt,PoW/PoS,0,0.000000
611,SixEleven,SHA-256,PoW,0,0.000000
808,808,SHA-256,PoW/PoS,0,0.000000


In [10]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)

In [11]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,CirculatingSupply
42,42 Coin,Scrypt,PoW/PoS,41.999952,41.999952
NSR,NuShares,PoS,PoS,6178769765.8373,0.000000
TRI,Triangles Coin,X13,PoW/PoS,199280.46296,0.000000
CMTC,CometCoin,Scrypt,PoW,872830,0.000000
CHAT,OpenChat,Scrypt,PoW/PoS,1000000000,0.000000


In [12]:
# Drop rows where there are 'N/A' text values
crypto_df.replace(to_replace='N/A',value=np.nan,inplace=True)
crypto_df.dropna(inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,CirculatingSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,41.999952
NSR,NuShares,PoS,PoS,6.178770e+09,0.000000
TRI,Triangles Coin,X13,PoW/PoS,1.992805e+05,0.000000
CMTC,CometCoin,Scrypt,PoW,8.728300e+05,0.000000
CHAT,OpenChat,Scrypt,PoW/PoS,1.000000e+09,0.000000


In [13]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
y = crypto_df[['CoinName']]
y.head()

,CoinName
42,42 Coin
NSR,NuShares
TRI,Triangles Coin
CMTC,CometCoin
CHAT,OpenChat


In [14]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns=['CoinName'],inplace=True)

In [15]:
# Create dummy variables for text features
y = pd.get_dummies(y)
crypto_df = pd.get_dummies(crypto_df)
y.head(), crypto_df.head()

(      CoinName_42 Coin  CoinName_AEON  CoinName_ARK  CoinName_Achain  \
 42                   1              0             0                0   
 NSR                  0              0             0                0   
 TRI                  0              0             0                0   
 CMTC                 0              0             0                0   
 CHAT                 0              0             0                0   
 
       CoinName_Acute Angle Cloud  CoinName_Aidos Kuneen  CoinName_Aion  \
 42                             0                      0              0   
 NSR                            0                      0              0   
 TRI                            0                      0              0   
 CMTC                           0                      0              0   
 CHAT                           0                      0              0   
 
       CoinName_Amber  CoinName_Anchor Protocol  CoinName_Aptcoin  ...  \
 42                 0             

In [16]:
# Standardize data
crypto_df_scaled = StandardScaler().fit_transform(crypto_df)

### Reducing Dimensions Using PCA

In [17]:
# Use PCA to reduce dimensions to 3 principal components
# Initialize PCA model
pca = PCA(n_components=3)

# Get principal components for data.
crypto_pca = pca.fit_transform(crypto_df_scaled)

In [18]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1","PC 2","PC 3"], 
)

pcs_df.set_index(crypto_df.index,inplace=True)
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.229940,1.084834,-1.437896
NSR,-0.124265,1.417266,-0.284618
TRI,-0.174187,1.869171,-1.747098
CMTC,-0.367993,-0.916143,-0.371213
CHAT,-0.229933,1.084833,-1.437896


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [19]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [20]:
# Initialize the K-Means model
k_model = KMeans(n_clusters=10, random_state=0)

# Fit the model
k_model.fit(pcs_df)

# Predict clusters
k_prediction = k_model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df['class'] = k_model.labels_

pcs_df.head()

,PC 1,PC 2,PC 3,class
42,-0.229940,1.084834,-1.437896,9
NSR,-0.124265,1.417266,-0.284618,6
TRI,-0.174187,1.869171,-1.747098,0
CMTC,-0.367993,-0.916143,-0.371213,8
CHAT,-0.229933,1.084833,-1.437896,9


In [21]:
# Create cluster df
cluster_prep_df = pd.DataFrame.from_dict(res_js['Data'],orient="index")
cluster_prep_df = cluster_prep_df[['CoinName','Algorithm','ProofType','TotalCoinsMined','CirculatingSupply']]

cluster_df = pd.merge(pcs_df,cluster_prep_df, left_index=True, right_index=True)
cluster_df.head()

,PC 1,PC 2,PC 3,class,CoinName,Algorithm,ProofType,TotalCoinsMined,CirculatingSupply
42,-0.229940,1.084834,-1.437896,9,42 Coin,Scrypt,PoW/PoS,41.999952,41.999952
NSR,-0.124265,1.417266,-0.284618,6,NuShares,PoS,PoS,6178769765.8373,0.000000
TRI,-0.174187,1.869171,-1.747098,0,Triangles Coin,X13,PoW/PoS,199280.46296,0.000000
CMTC,-0.367993,-0.916143,-0.371213,8,CometCoin,Scrypt,PoW,872830,0.000000
CHAT,-0.229933,1.084833,-1.437896,9,OpenChat,Scrypt,PoW/PoS,1000000000,0.000000


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [22]:
mn_scaler = MinMaxScaler()
cluster_plt_df = mn_scaler.fit_transform(cluster_df[['TotalCoinsMined','CirculatingSupply']])

In [23]:
cluster_prep_df = pd.DataFrame(cluster_plt_df,columns=['TotalCoinsMined','CirculatingSupply'],index=cluster_df.index)
cluster_prep_df.head()

,TotalCoinsMined,CirculatingSupply
42,0.000000e+00,4.506293e-14
NSR,6.241182e-06,0.000000e+00
TRI,2.012510e-10,0.000000e+00
CMTC,8.816040e-10,0.000000e+00
CHAT,1.010101e-06,0.000000e+00


In [24]:
cluster_prep_df['CoinName'] = cluster_df['CoinName']
cluster_prep_df['class'] = cluster_df['class']
cluster_prep_df.head()

,TotalCoinsMined,CirculatingSupply,CoinName,class
42,0.000000e+00,4.506293e-14,42 Coin,9
NSR,6.241182e-06,0.000000e+00,NuShares,6
TRI,2.012510e-10,0.000000e+00,Triangles Coin,0
CMTC,8.816040e-10,0.000000e+00,CometCoin,8
CHAT,1.010101e-06,0.000000e+00,OpenChat,9


In [32]:
# create scatter plot
cluster_prep_df.hvplot.scatter(x="TotalCoinsMined", y="CirculatingSupply", by="class",hover_cols=["CoinName"])

:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (CirculatingSupply,CoinName)

#### Table of Tradable Cryptocurrencies

In [26]:
# Table with tradable cryptos
cluster_table = cluster_df.hvplot.table(columns=['CoinName','Algorithm','ProofType','TotalCoinsMined','CirculatingSupply','class'], sortable=True, selectable=True)
cluster_table

:Table   [CoinName,Algorithm,ProofType,TotalCoinsMined,CirculatingSupply,class]

In [29]:
# Print the total number of tradable cryptocurrencies
print(f"There are {cluster_df.shape[0]} tradeable cryptocurrencies")

There are 140 tradeable cryptocurrencies
